In [44]:
import numpy as np
import pandas as pd
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [45]:
for dirname, _, filenames in os.walk('./titanic_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./titanic_data/my_submission.csv
./titanic_data/test.csv
./titanic_data/train.csv
./titanic_data/gender_submission.csv


In [46]:
train_data = pd.read_csv('./titanic_data/train.csv')
test_data = pd.read_csv('./titanic_data/test.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
y = train_data['Survived']

features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
print(X.head())

   Pclass   Age  SibSp  Parch     Fare  Sex_female  Sex_male  Embarked_C  \
0       3  22.0      1      0   7.2500       False      True       False   
1       1  38.0      1      0  71.2833        True     False        True   
2       3  26.0      0      0   7.9250        True     False       False   
3       1  35.0      1      0  53.1000        True     False       False   
4       3  35.0      0      0   8.0500       False      True       False   

   Embarked_Q  Embarked_S  
0       False        True  
1       False       False  
2       False        True  
3       False        True  
4       False        True  


In [48]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('titanic_data/my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [54]:
importances = pd.Series(model.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)
print("\nFeature importances:")
print(importances)


Feature importances:
Sex_male      0.273960
Sex_female    0.251928
Pclass        0.132458
Fare          0.128738
Age           0.108647
SibSp         0.042303
Parch         0.027802
Embarked_S    0.014688
Embarked_C    0.012705
Embarked_Q    0.006771
dtype: float64
